# Breakout
# Using Deep Q-Learning method. 
http://web.ift.uib.no/Teori/KURS/WRK/TeX/symALL.html

DQN paper:

https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf

https://pdfs.semanticscholar.org/presentation/a210/9d4f149e1526907fc0c907458656bca6f73e.pdf

# Experience replay (off policy, random sample)

breack the relation of updating weight. (two methods)

1:Experience replay (off policy, random sample)

2:Fixed Q-targets (two network !!! new $Q_{value}$ and old $Q_{target}$ )

   old $Q_{target}$ is using for update network. $R+r*maxQ'$
   
   new Q is $Q_{value}$.
   
   $NN = nn + \alpha * ( Q_{target} - Q_{value} )$

### Loadding library

In [1]:
import gym
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import prettytensor as pt
import scipy.ndimage
import time
import sys
import reinforcement_learning as rl
from reinforcement_learning import ReplayMemory

RENDER = False  # rendering wastes time


/home/yanhua/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Create environment

In [2]:
env = gym.make('Breakout-v0')
env.seed(1)     # reproducible, general Policy gradient has high variance
env = env.unwrapped

MEMORY_SIZE = 3000
ACTION_SPACE = 11

# The number of possible actions that the agent may take in every step.
num_actions = env.action_space.n


environment information : action

In [3]:
print(env.action_space)

Discrete(4)


environment information : observation (image)

In [4]:
print(env.observation_space)

Box(210, 160, 3)


# Create neural network

In [5]:
!rm /home/yanhua/Documents/jupyter/yan/demo/Deep_Q_Network/checkpoints_tutorial/Breakout-v0/*

rm: cannot remove '/home/yanhua/Documents/jupyter/yan/demo/Deep_Q_Network/checkpoints_tutorial/Breakout-v0/*': No such file or directory


In [6]:
#rl.checkpoint_base_dir = '/home/yanhua/Documents/jupyter/TensorFlow-Tutorials-master/checkpoints_tutorial16/'
rl.checkpoint_base_dir = '/home/yanhua/Documents/jupyter/yan/demo/Deep_Q_Network/checkpoints_tutorial/'

In [7]:
env_name = 'Breakout-v0'
rl.update_paths(env_name=env_name)

In [8]:
#model = rl.NeuralNetwork(num_actions=num_actions,replay_memory= None)
model = rl.NeuralNetwork(num_actions=num_actions,replay_memory= rl.ReplayMemory(size=200000,num_actions=num_actions))

Trying to restore last checkpoint ...
Failed to restore checkpoint from: /home/yanhua/Documents/jupyter/yan/demo/Deep_Q_Network/checkpoints_tutorial/Breakout-v0
Initializing variables instead.


In [9]:
model.get_weights_variable(layer_name='layer_conv1')

<tf.Variable 'layer_conv1/weights:0' shape=(3, 3, 2, 16) dtype=float32_ref>

cd /home/yanhua/Documents/jupyter/yan/demo/Deep_Q_Network/

tensorboard --logdir=/home/yanhua/Documents/jupyter/yan/demo/Deep_Q_Network/logs

http://0.0.0.0:6006/

convolution neural network 3 layer + 4 dense layer 


The formula for updating the Q-value is:

$$
    Q(s_{t},a_{t}) \leftarrow \underbrace{r_{t}}_{\rm reward} + \underbrace{\gamma}_{\rm discount} \cdot \underbrace{\max_{a}Q(s_{t+1}, a)}_{\rm estimate~of~future~rewards}
$$

### loss = q_values.l2_regression(target=q_values_new)


# data pre-processing

## Testing the initial neural network

num_episodes = 1

In [10]:
count_episodes = 0
num_episodes = 1
end_episode = True
count_states = model.get_count_states()
training=False

In [11]:
epsilon_greedy = rl.EpsilonGreedy(start_value=1.0,
                                            end_value=0.1,
                                            num_iterations=1e6,
                                            num_actions=num_actions,
                                            epsilon_testing=0.01)

In [12]:
while count_episodes <= num_episodes:
    if end_episode:
        # Reset the game-environment and get the first image-frame.
        img = env.reset()
        # Create a new motion-tracer for processing images from the
        # game-environment. Initialize with the first image-frame.
        # This resets the motion-tracer so the trace starts again.
        # This could also be done if end_life==True.
        motion_tracer = rl.MotionTracer(img)
        # Reset the reward for the entire episode to zero.
        # This is only used for printing statistics.
        reward_episode = 0.0

        # Increase the counter for the number of episodes.
        # This counter is stored inside the TensorFlow graph
        # so it can be saved and restored with the checkpoint.
        #count_episodes = model.increase_count_episodes()
        count_episodes = count_episodes + 1
        # Get the number of lives that the agent has left in this episode.
        num_lives = env.unwrapped.ale.lives()

    # Get the state of the game-environment from the motion-tracer.
    # The state has two images: (1) The last image-frame from the game
    # and (2) a motion-trace that shows movement trajectories.
    state = motion_tracer.get_state()
    # Use the Neural Network to estimate the Q-values for the state.
    # Note that the function assumes an array of states and returns
    # a 2-dim array of Q-values, but we just have a single state here.
    q_values = model.get_q_values(states=[state])[0]
    
    action, epsilon = epsilon_greedy.get_action(q_values=q_values,
                                                             iteration=count_states,
                                                             training=training)
    # Take a step in the game-environment using the given action.
    # Note that in OpenAI Gym, the step-function actually repeats the
    # action between 2 and 4 time-steps for Atari games, with the number
    # chosen at random.
    img, reward, end_episode, info = env.step(action)
    # Process the image from the game-environment in the motion-tracer.
    # This will first be used in the next iteration of the loop.
    motion_tracer.process(image=img)
    # Add the reward for the step to the reward for the entire episode.
    reward_episode += reward

    
    # Render the game-environment to screen.
    env.render()

    # Insert a small pause to slow down the game,
    # making it easier to follow for human eyes.
    time.sleep(0.005)
    

# Training neural network

In [13]:
replay_fraction = rl.LinearControlSignal(start_value=0.1,end_value=1.0,num_iterations=5e6)

# The learning-rate for the optimizer decreases linearly.
learning_rate_control = rl.LinearControlSignal(start_value=1e-3,end_value=1e-5,num_iterations=5e6)

# The loss-limit is used to abort the optimization whenever the
# mean batch-loss falls below this limit.
loss_limit_control = rl.LinearControlSignal(start_value=0.1,end_value=0.015,num_iterations=5e6)

# The maximum number of epochs to perform during optimization.
# This is increased from 5 to 10 epochs, because it was found for
max_epochs_control = rl.LinearControlSignal(start_value=5.0,end_value=10.0,num_iterations=5e6)


In [14]:
count_episodes = 0
num_episodes = 100
end_episode = True
count_states = model.get_count_states()
training=True

## trained neural network model about 80 hours

$$
    Q(state_{t},NOOP) \leftarrow \underbrace{r_{t}}_{\rm reward} + \underbrace{\gamma}_{\rm discount} \cdot \underbrace{\max_{a}Q(state_{t+1}, a)}_{\rm estimate~of~future~rewards} = 1.0 + 0.97 \cdot 1.830 \simeq 2.775
$$


In [15]:
model.replay_memory.num_used

0

In [16]:
while count_episodes <= num_episodes:
    if end_episode:
        # Reset the game-environment and get the first image-frame.
        img = env.reset()
        # Create a new motion-tracer for processing images from the
        # game-environment. Initialize with the first image-frame.
        # This resets the motion-tracer so the trace starts again.
        # This could also be done if end_life==True.
        motion_tracer = rl.MotionTracer(img)
        # Reset the reward for the entire episode to zero.
        # This is only used for printing statistics.
        reward_episode = 0.0

        # Increase the counter for the number of episodes.
        # This counter is stored inside the TensorFlow graph
        # so it can be saved and restored with the checkpoint.
        count_episodes = count_episodes + 1
        # Get the number of lives that the agent has left in this episode.
        num_lives = env.unwrapped.ale.lives()

    # Get the state of the game-environment from the motion-tracer.
    # The state has two images: (1) The last image-frame from the game
    # and (2) a motion-trace that shows movement trajectories.
    state = motion_tracer.get_state()
    # Use the Neural Network to estimate the Q-values for the state.
    # Note that the function assumes an array of states and returns
    # a 2-dim array of Q-values, but we just have a single state here.

    q_values = model.get_q_values(states=[state])[0]
    
    action, epsilon = epsilon_greedy.get_action(q_values=q_values,
                                                             iteration=count_states,
                                                             training=training)
    # Take a step in the game-environment using the given action.
    # Note that in OpenAI Gym, the step-function actually repeats the
    # action between 2 and 4 time-steps for Atari games, with the number
    # chosen at random.
    img, reward, end_episode, info = env.step(action)
    # Process the image from the game-environment in the motion-tracer.
    # This will first be used in the next iteration of the loop.
    motion_tracer.process(image=img)
    # Add the reward for the step to the reward for the entire episode.
    reward_episode += reward
    
    # Determine if a life was lost in this step.
    num_lives_new = env.unwrapped.ale.lives()
    end_life = (num_lives_new < num_lives)
    num_lives = num_lives_new

    count_states = model.increase_count_states()

    # Add the state of the game-environment to the replay-memory.
    model.replay_memory.add(state=state,
                                       q_values=q_values,
                                       action=action,
                                       reward=reward,
                                       end_life=end_life,
                                       end_episode=end_episode)

    # How much of the replay-memory should be used.
    use_fraction = replay_fraction.get_value(iteration=count_states)

        
    if end_episode:
        msg = "{0:4}:\t Epsilon: {1:4.2f}\t Reward: {2:.1f}"
        print(msg.format(count_episodes, epsilon,
                                 reward_episode))
    
    # When the replay-memory is sufficiently full.
#    if replay_memory.is_full() \
#                    or replay_memory.used_fraction() > use_fraction or count_episodes == num_episodes:
    if model.replay_memory.is_full() \
                    or (count_episodes == num_episodes and end_episode):

                    # Update all Q-values in the replay-memory through a backwards-sweep.
        model.replay_memory.update_all_q_values()

        # Get the control parameters for optimization of the Neural Network.
        # These are changed linearly depending on the state-counter.
        learning_rate = learning_rate_control.get_value(iteration=count_states)
        loss_limit = loss_limit_control.get_value(iteration=count_states)
        max_epochs = max_epochs_control.get_value(iteration=count_states)

        # Perform an optimization run on the Neural Network so as to
        # improve the estimates for the Q-values.
        # This will sample random batches from the replay-memory.
        
        model.optimize(learning_rate=learning_rate,
                                        loss_limit=loss_limit,
                                        max_epochs=max_epochs)
        
        # Save a checkpoint of the Neural Network so we can reload it.
#        self.model.save_checkpoint(count_states)

        # Reset the replay-memory. This throws away all the data we have
        # just gathered, so we will have to fill the replay-memory again.
        model.replay_memory.reset()


   1:	 Epsilon: 1.00	 Reward: 0.0
   2:	 Epsilon: 1.00	 Reward: 0.0
   3:	 Epsilon: 1.00	 Reward: 2.0
   4:	 Epsilon: 1.00	 Reward: 1.0
   5:	 Epsilon: 1.00	 Reward: 0.0
   6:	 Epsilon: 1.00	 Reward: 2.0
   7:	 Epsilon: 1.00	 Reward: 3.0
   8:	 Epsilon: 1.00	 Reward: 2.0
   9:	 Epsilon: 1.00	 Reward: 1.0
  10:	 Epsilon: 1.00	 Reward: 1.0
  11:	 Epsilon: 1.00	 Reward: 8.0
  12:	 Epsilon: 1.00	 Reward: 2.0
  13:	 Epsilon: 1.00	 Reward: 1.0
  14:	 Epsilon: 1.00	 Reward: 2.0
  15:	 Epsilon: 1.00	 Reward: 4.0
  16:	 Epsilon: 1.00	 Reward: 0.0
  17:	 Epsilon: 1.00	 Reward: 0.0
  18:	 Epsilon: 1.00	 Reward: 1.0
  19:	 Epsilon: 1.00	 Reward: 2.0
  20:	 Epsilon: 1.00	 Reward: 2.0
  21:	 Epsilon: 1.00	 Reward: 2.0
  22:	 Epsilon: 0.99	 Reward: 2.0
  23:	 Epsilon: 0.99	 Reward: 0.0
  24:	 Epsilon: 0.99	 Reward: 2.0
  25:	 Epsilon: 0.99	 Reward: 1.0
  26:	 Epsilon: 0.99	 Reward: 1.0
  27:	 Epsilon: 0.99	 Reward: 0.0
  28:	 Epsilon: 0.99	 Reward: 4.0
  29:	 Epsilon: 0.99	 Reward: 0.0
  30:	 Epsilon

store the memory (image, reward, etc...) for the training

In [17]:
model.replay_memory.states.shape,model.replay_memory.q_values.shape,model.replay_memory.actions.shape,model.replay_memory.end_episode.shape,model.replay_memory.end_life.shape

((200000, 105, 80, 2), (200000, 4), (200000,), (200000,), (200000,))

In [18]:
model.replay_memory.rewards[0:10000]

array([0., 0., 0., ..., 0., 0., 0.])

In [19]:
model.replay_memory.rewards[0:10000]

array([0., 0., 0., ..., 0., 0., 0.])

# Testing the trained neural network

In [20]:
count_episodes = 0
num_episodes = 3
end_episode = True

In [21]:
while count_episodes <= num_episodes:
    if end_episode:
        # Reset the game-environment and get the first image-frame.
        img = env.reset()
        # Create a new motion-tracer for processing images from the
        # game-environment. Initialize with the first image-frame.
        # This resets the motion-tracer so the trace starts again.
        # This could also be done if end_life==True.
        motion_tracer = rl.MotionTracer(img)
        # Reset the reward for the entire episode to zero.
        # This is only used for printing statistics.
        reward_episode = 0.0

        # Increase the counter for the number of episodes.
        # This counter is stored inside the TensorFlow graph
        # so it can be saved and restored with the checkpoint.
        #count_episodes = model.increase_count_episodes()
        count_episodes = count_episodes + 1
        # Get the number of lives that the agent has left in this episode.
        num_lives = env.unwrapped.ale.lives()

    # Get the state of the game-environment from the motion-tracer.
    # The state has two images: (1) The last image-frame from the game
    # and (2) a motion-trace that shows movement trajectories.
    state = motion_tracer.get_state()
    # Use the Neural Network to estimate the Q-values for the state.
    # Note that the function assumes an array of states and returns
    # a 2-dim array of Q-values, but we just have a single state here.
    q_values = model.get_q_values(states=[state])[0]
    
    action, epsilon = epsilon_greedy.get_action(q_values=q_values,
                                                             iteration=count_states,
                                                             training=training)
    # Take a step in the game-environment using the given action.
    # Note that in OpenAI Gym, the step-function actually repeats the
    # action between 2 and 4 time-steps for Atari games, with the number
    # chosen at random.
    img, reward, end_episode, info = env.step(action)
    # Process the image from the game-environment in the motion-tracer.
    # This will first be used in the next iteration of the loop.
    motion_tracer.process(image=img)
    # Add the reward for the step to the reward for the entire episode.
    reward_episode += reward

    
    # Render the game-environment to screen.
    env.render()

    # Insert a small pause to slow down the game,
    # making it easier to follow for human eyes.
    time.sleep(0.01)
    # Print Q-values and reward to screen.
    if(end_life or end_episode):
        msg = "{0:4}:\tQ-min: {1:5.3f}\tQ-max: {2:5.3f}\tLives: {3}\tReward: {4:.1f}"
        print(msg.format(count_episodes, np.min(q_values),np.max(q_values), num_lives, reward_episode))

   1:	Q-min: 0.007	Q-max: 0.021	Lives: 5	Reward: 1.0
   2:	Q-min: 0.030	Q-max: 0.040	Lives: 5	Reward: 0.0
   3:	Q-min: 0.005	Q-max: 0.019	Lives: 5	Reward: 0.0


# Save the trained neural network.

In [22]:
count_states

24490

In [23]:
model.checkpoint_path

'/home/yanhua/Documents/jupyter/yan/demo/Deep_Q_Network/checkpoints_tutorial/Breakout-v0/checkpoint'

In [24]:
model.save_checkpoint(count_states)

Saved checkpoint.


In [25]:
model.saver.save(model.session,save_path='/home/yanhua/Documents/jupyter/yan/demo/Deep_Q_Network/checkpoints_tutorial/checkporint/breakout/',global_step=count_states)
print("Saved checkpoint.")

Saved checkpoint.


In [26]:
ls /home/yanhua/Documents/jupyter/yan/demo/Deep_Q_Network/checkpoints_tutorial/Breakout-v0

checkpoint                            checkpoint-24490.index
checkpoint-24490.data-00000-of-00001  checkpoint-24490.meta


## Using the Pre-trained neural network

### The model is trained about 80 hours (3 days) 

restart kernel

In [1]:
import gym
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import prettytensor as pt
import scipy.ndimage
import time
import sys
import reinforcement_learning as rl
from reinforcement_learning import ReplayMemory
env = gym.make('Breakout-v0')
env.seed(1)     # reproducible, general Policy gradient has high variance
env = env.unwrapped
MEMORY_SIZE = 3000
ACTION_SPACE = 11
# The number of possible actions that the agent may take in every step.
num_actions = env.action_space.n
epsilon_greedy = rl.EpsilonGreedy(start_value=1.0,
                                            end_value=0.1,
                                            num_iterations=1e6,
                                            num_actions=num_actions,
                                            epsilon_testing=0.01)
rl.checkpoint_base_dir = '/home/yanhua/Documents/jupyter/TensorFlow-Tutorials-master/checkpoints_tutorial16/'
env_name = 'Breakout-v0'
rl.update_paths(env_name=env_name)
model = rl.NeuralNetwork(num_actions=num_actions,replay_memory= rl.ReplayMemory(size=200000,num_actions=num_actions))
count_states = model.get_count_states()
training=False

/home/yanhua/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Trying to restore last checkpoint ...
INFO:tensorflow:Restoring parameters from /home/yanhua/Documents/jupyter/TensorFlow-Tutorials-master/checkpoints_tutorial16/Breakout-v0/checkpoint-253779762
Restored checkpoint from: /home/yanhua/Documents/jupyter/TensorFlow-Tutorials-master/checkpoints_tutorial16/Breakout-v0/checkpoint-253779762


In [4]:
count_episodes = 0
num_episodes = 2
end_episode = True

In [5]:
while count_episodes <= num_episodes:
    if end_episode:
        # Reset the game-environment and get the first image-frame.
        img = env.reset()
        # Create a new motion-tracer for processing images from the
        # game-environment. Initialize with the first image-frame.
        # This resets the motion-tracer so the trace starts again.
        # This could also be done if end_life==True.
        motion_tracer = rl.MotionTracer(img)
        # Reset the reward for the entire episode to zero.
        # This is only used for printing statistics.
        reward_episode = 0.0

        # Increase the counter for the number of episodes.
        # This counter is stored inside the TensorFlow graph
        # so it can be saved and restored with the checkpoint.
        #count_episodes = model.increase_count_episodes()
        count_episodes = count_episodes + 1
        # Get the number of lives that the agent has left in this episode.
        num_lives = env.unwrapped.ale.lives()

    # Get the state of the game-environment from the motion-tracer.
    # The state has two images: (1) The last image-frame from the game
    # and (2) a motion-trace that shows movement trajectories.
    state = motion_tracer.get_state()
    # Use the Neural Network to estimate the Q-values for the state.
    # Note that the function assumes an array of states and returns
    # a 2-dim array of Q-values, but we just have a single state here.
    q_values = model.get_q_values(states=[state])[0]
    
    action, epsilon = epsilon_greedy.get_action(q_values=q_values,
                                                             iteration=count_states,
                                                             training=training)
    # Take a step in the game-environment using the given action.
    # Note that in OpenAI Gym, the step-function actually repeats the
    # action between 2 and 4 time-steps for Atari games, with the number
    # chosen at random.
    img, reward, end_episode, info = env.step(action)
    # Process the image from the game-environment in the motion-tracer.
    # This will first be used in the next iteration of the loop.
    motion_tracer.process(image=img)
    # Add the reward for the step to the reward for the entire episode.
    reward_episode += reward

    
    # Render the game-environment to screen.
    env.render()

    # Insert a small pause to slow down the game,
    # making it easier to follow for human eyes.
    time.sleep(0.005)
    # Print Q-values and reward to screen.
    if(end_episode):
        msg = "{0:4}:\tQ-min: {1:5.3f}\tQ-max: {2:5.3f}\tLives: {3}\tReward: {4:.1f}"
        print(msg.format(count_episodes, np.min(q_values),np.max(q_values), num_lives, reward_episode))

   1:	Q-min: -0.335	Q-max: -0.235	Lives: 5	Reward: 98.0
   2:	Q-min: -0.420	Q-max: 0.043	Lives: 5	Reward: 59.0
